# Data Mining for Business Analytics

## Similarity, Neighbors

Spring 2019 - Prof. George Valkanas

Material based on content courtesy of Prof. Foster Provost

***


## Clustering

We already saw how we can use _similarity_ between two instances to produce recommendations, even in the absence of a target variable.

_Similarity_ is also a key element in **clustering**, i.e., the generation of _natural groups_ of our instances. We have already seen that different similarity measures result in different _rankings_ of the same set of instances. With that in mind, the _similarity_ that we use greatly affects the result of our **clustering** algorithms.


Below, we discuss 2 different clustering techniques:
* $k$-Means
* Hierarchical Clustering

Through our discussion, you need to be able to understand:
1. How **clustering** differs from **classification**
1. What is _the result_ of a clustering algorithm
1. How $k$-Means works
1. How hierarchical clustering works

***

Take a look at the following image

<img src="images/raw_data.png" />


How many _clusters_ / "natural groupings" of the data do you see?


Take a look at the same image, where the data points are associated with one of two classes: Red or Blue.

<img src="images/class_data.png" />

During _classification_ , our data points (instances) come with those _predefined_ classes. The class value of an instance appears in the _target variable_.

***

**Question:** Using the two earlier images and the brief explanations above, how do _you_ think that clustering differs from classification?


**Answer:** (write down your thoughts)

***

In [ ]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = 10, 8


## KMeans

The "classic" approach for findings clusters is via the **$k$-Means algorithm**, which will find a set of $k$ clusters.

The value $k$ is a **parameter** to the model, i.e. **we** must provide the number of clusters that we expect the algorithm to find.

**Question:** _What_ is a good $k$ value for the algorithm?


Here's a video of how the algorithm works: https://www.youtube.com/watch?v=BVFG7fd1H30


The $k$-Means algorithm is implemented in all major (self-respecting) data mining libraries. It's also available under **sklearn.cluster**.



In [ ]:
# Let's read our Whiskey data again
data = pd.read_csv("data/scotch.csv")
data = data.drop([u'age', u'dist', u'score', u'percent', u'region', u'district', u'islay', u'midland', u'spey', u'east', u'west', u'north ', u'lowland', u'campbell', u'islands'], axis=1)
data.head()

In [ ]:
# Let's train a kMeans model over the whiskeys. We want 6 clusters.
k_clusters = 6

## Fit clusters like in our previous models/transformations/standarization 
## (e.g. Logistic, Vectorization,...)

model = KMeans(k_clusters)
model.fit(data)


***

What clusters do we get? Let's get "predictions" of the model. To do that, we will use the `predict()` method of the kmeans cluster.

**Question:** Clustering is an **unsupervised** task. What do you think we mean by "prediction" here then?

-

-

-

-

**Question:** How do we compute this distance between an instance and a cluster?


In [ ]:
print ("Records in our dataset (rows): ", len(data.index))
print ("Then we predict one cluster per record, which means length of: ", len(model.predict(data)) )


In [ ]:
# The index index contains the names of the whiskeys that we know of
data.index

In [ ]:
clusters = model.predict(data)

clusters

What do you think that the above numbers mean?
***

Let's print each instance with its associated cluster **and** the cluster _centroids_

In [ ]:
l = []
data_predicts = model.predict(data)
for i, cid in zip(data.index, data_predicts):
    row_list = [i, cid]
    row_list.extend(model.cluster_centers_[cid])
    l.append(row_list)

colNames = ['Whiskey','Cluster_predicted']
colNames.extend( [ 'Cluster ' + c for c in data.columns ]  )
pd.DataFrame( l, columns=colNames )[0:10]


Let's put each cluster into its own column!


In [ ]:

cluster_listing = {}
for cluster in range(k_clusters):
    cluster_name = 'Cluster ' + str(cluster)
    cluster_listing[cluster_name] = [''] * len(data)
    where_in_cluster = np.where(clusters == cluster)[0]
    cluster_listing[cluster_name][0:len(where_in_cluster)] = data.index[where_in_cluster]

# Print clusters
pd.DataFrame(cluster_listing).loc[0:np.max(np.bincount(clusters)) - 1,:]


What do you think, to the results make sense?

## Another clustering application, closer to home (all puns intended)

Well, we can cluster whiskeys, we can cluster wines and we can cluster documents (e.g., news articles), but whether the results are particularly meaningful or not is often open for interpretation and left as follow-up work for the data scientist, in collaboration with a domain expert (i.e., a wine taster perhaps?).

So, let's work with some data that we can get a better understanding of: _locations_ ! 


Under the `data/` directory, there is a file with GPS coordinates in it. The file contains several lines, each of which is a GPS signal (longitude, latitude). Let's read the file and see what it's about.  The file uses the comma `','` as a separator between the columns, so we'll specify that. The data is also in decimal format (`float`).

*** 

In [ ]:
coordinates = pd.read_csv( "data/locations.csv", sep=',', dtype=float )
coordinates.head()

Instead of me telling you what these locations are, let's actually show them on a figure.

In [ ]:
fig = plt.figure(figsize=(7, 7))  # Small plot at first
plt.scatter( coordinates['long'], coordinates['lat'], c='black', s=(72./fig.dpi)**2 )  # The value next to 's' is the size of a pixel!
plt.show()

What is this blob!? We can't make anything out of it! OK, it seems that most of the information is centered around some specific ranges though.

So, let's plot again by focusing on these ranges and we'll blow up the image a bit more. For the ranges, we'll use the `xlim()` and `ylim()` methods of the matplotlib library.

Try it in the code below!

In [ ]:
fig = plt.figure(figsize=(20, 20))  # Let's create a bigger picture

plt.scatter( coordinates['long'], coordinates['lat'], c='black', s=(72./fig.dpi)**2 )  # The value next to 's' is the size of a pixel!

# Pick the ranges that will help you figure out what the shape is about
plt.xlim( None, None )  # That's for the x-coordinate range. Format is: x_min, x_max
plt.ylim( None, None )  # That's for the y-coordinate range. Format is: x_min, x_max

plt.show()

Aha! Now it starts to make sense. OK, let's use these coordinates to actually find some clusters of the coordinates.

Now, $k$-Means is fast, but it's not "classroom demo" fast. For that reason, we'll _sample_ our coordinates and use only 10000 of them. You can try with the entire dataset on your own.

In [ ]:
sample_coords = coordinates.sample(10000)

In [ ]:
# Let's create 30 clusters. You can try different values and see how the results change.
k_clusters = 30

model = KMeans(k_clusters)

# Train the model on the sample coordinates, not the original.
model.fit(sample_coords)

In [ ]:
# Let's predict the cluster that each coordinate belongs to!
# Notice that we are now doing this for ALL coordinates
predict_ids = model.predict(coordinates)

In [ ]:
# Transpose the matrix: get every row as a column and every column as a row
# We do this so that we can pass the information to the scatter plot below.
clust_coords = model.cluster_centers_.T

fig = plt.figure(figsize=(20, 20))
plt.scatter( coordinates['long'], coordinates['lat'], c=predict_ids, s=(72./fig.dpi)**2, cmap='tab20' )
plt.scatter( clust_coords[0], clust_coords[1], c='black', s=50, marker='*' )

# Use the coordinates that you had earlier, to make the plot look nice!
plt.xlim( None, None )
plt.ylim( None, None )

plt.show()

What do you think? Do the star-shaped points make _some_ sense? Do the colored regions make sense?

### <div style="color: red">A note on visualization</div>

The above exercise should show the importance of _visualizing_ information for several reasons.

- First, it is more intuitive and easier to explain -- an image is said to be worth 1000 words after all! There is, in fact, an entire science / discipline around visualizing information.

- Second, this type of visualization allows us to see issues that we have with our data. Can you spot any?


## Hierarchical Clustering

kMeans is just one way to find similar groups, but not the only one. Another very common method is Hierarchical Clustering.

First let's look at a simple example to illustrate.  Given a set of records (A-F) with two features, we can visualize them on a 2 dimensional surface.  Clustering proceeds as follows.  First consider each point to be its own cluster.  Then, iteratively, group together the closest two clusters.  In the figure, circles were drawn in order of grouping.  The second diagram is a visualization of the hierarchy of groupings, called a "dendrogram."  You can clip it at any point, vertically, and get "the best" clustering for a certain number of groups.


<img src="images/cutting.png" height=40% width=40%>

Here is a visualization of a part of the dendrogram for the whiskey clustering in the book:

<img src="images/cross_section.png" height=70% width=70%>

***



### Dendrograms in code

We'll go back to our Whiskey example to illustrate the dendrogram. We will use the module **scipy.cluster.hierarchy** for this purpose.

The approach is as follows:
1. We start by computing the distance between every pair of whiskeys
1. Then, we will use the `linkage()` method coming from the `scipy.cluster.hierarchy` module. This method will actually perform the hierarchical clustering we want! Notice the difference between this approach and creating the k-Means model. More information about the method is available [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html)
1. We then pass the resulting hierarchical clustering to the `dendrogram()` method, which will visualize the information in the way that we described earlier.

In [ ]:
# This function gets pairwise distances between observations in n-dimensional space.
dists = distance.pdist(data, metric="cosine")

# This scipy's function performs hierarchical/agglomerative clustering on the condensed distance matrix y.
links = linkage(dists, method='average')

# Now we want to plot those 'links' using "dendrogram" function
# ATTENTION: This line must come BEFORE the dendrogram() method below!!!
plt.rcParams['figure.figsize'] = 32, 16

den = dendrogram(links)

plt.xlabel('Samples',fontsize=18)
plt.ylabel('Distance',fontsize=18)
plt.xticks(rotation=90,fontsize=16)
plt.show()


We can use other measures:

In [ ]:
# This function gets pairwise distances between observations in n-dimensional space.
dists = distance.pdist(data, metric="euclidean")

# This scipy's function performs hierarchical/agglomerative clustering on the condensed distance matrix y.
links = linkage(dists, method='average')

# Now we want to plot those 'links' using "dendrogram" function
plt.rcParams['figure.figsize'] = 32, 16

den = dendrogram(links)

plt.xlabel('Samples',fontsize=18)
plt.ylabel('Distance',fontsize=18)
plt.xticks(rotation=90,fontsize=16)
plt.show()


#### Question

Clusters **do not** come with predetermined labels. Can you think of an approach to "characterize them" ? Can you think of an _automated approach_ to characterize them?
***

#### Question

Both $k$-Means and Hierarchical Clustering, as described earlier perform **hard clustering**. What do you think that means?

***

### Clustering Text

#### (Very Briefly)

Clustering documents (e.g., articles) is a very common task, as it allows us to (very) quickly understand what the corpus (i.e., the entire set of documents) is about. In other words, the task is to figure out the various **topics** discussed in the corpus.

For this type of problem, instead of $k$-Means, we typically use another technique, called  **Latent Dirichlet allocation** (LDA). LDA works differently from $k$-Means, but in the end we still obtain a set of "clusters" / topics, which are characterized by words that appear in the document. 

Just like $k$-Means, LDA expects a parameter $k$ for the number of topics to discover.

Unlike $k$-Means, LDA does _soft_ clustering. What do you think this means?

***